# Data Wrangling

In [1]:
# Import packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import helper
import importlib
_ = importlib.reload(helper)


Let's read the data into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) so that we can begin to understand it.

*Note, we'll set `error_bad_lines=False` when reading the file in as there appear to be a very small number of records which would create a problem otherwise.*

In [2]:
dataset_name = 'ml-100k'
df_interactions = helper.get_csv(dataset_name, filename="interactions.csv.gz")
df_interactions.head()


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [3]:
df_interactions_cleaned = helper.get_csv(dataset_name, filename="interactions_cleaned.csv.gz")
df_interactions_cleaned.head()


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3.0,881250949
1,305,242,5.0,886307828
2,6,242,4.0,883268170
3,234,242,4.0,891033261
4,63,242,3.0,875747190


Next, we'll number each user and item, giving them their own sequential index.  This will allow us to hold the information in a sparse format where the sequential indices indicate the row and column in our ratings matrix.

In [4]:
users = df_interactions['USER_ID'].value_counts()
items = df_interactions['ITEM_ID'].value_counts()


In [5]:
df_user_index = pd.DataFrame({'USER_ID': users.index, '_USER_IDX': np.arange(users.shape[0])})

df_item_index = pd.DataFrame({'ITEM_ID': items.index, 
                              '_ITEM_IDX': np.arange(items.shape[0])})
helper.put_csv(df_user_index, dataset_name, "user_index.csv.gz")
helper.put_csv(df_item_index, dataset_name, "item_index.csv.gz")


In [6]:
df_interactions_cleaned = df_interactions_cleaned.merge(df_user_index).merge(df_item_index)
df_interactions_cleaned


,USER_ID,ITEM_ID,RATING,TIMESTAMP,_USER_IDX,_ITEM_IDX
0,196,242,3.0,881250949,646,287
1,305,242,5.0,886307828,121,287
2,6,242,4.0,883268170,134,287
3,234,242,4.0,891033261,8,287
4,63,242,3.0,875747190,378,287
...,...,...,...,...,...,...
99282,916,1597,3.0,880845206,44,1313
99283,828,1597,3.0,891037813,348,1313
99284,833,1597,5.0,875225193,81,1313
99285,425,1597,3.0,878738596,142,1313


### Prepare

Let's start by splitting in training and test sets.  This will allow us to estimate the model's accuracy on videos our customers rated, but wasn't included in our training.

In [7]:
df_test = df_interactions_cleaned.groupby('USER_ID').last().reset_index()
df_test


,USER_ID,ITEM_ID,RATING,TIMESTAMP,_USER_IDX,_ITEM_IDX
0,1,267,4.0,875692955,76,1182
1,2,296,3.0,888550871,487,1266
2,3,352,2.0,889237055,535,848
3,4,361,5.0,892002353,833,1128
4,5,376,2.0,879198045,184,876
...,...,...,...,...,...,...
938,939,1190,5.0,880260883,573,1112
939,940,355,1.0,889480552,335,691
940,941,919,5.0,875048887,885,349
941,942,969,4.0,891282817,411,436


In [8]:
helper.put_csv(df_test, dataset_name, "interactions_test.csv.gz")



In [9]:
df_train = df_interactions_cleaned.merge(df_test[['USER_ID', 'ITEM_ID']], 
                            on=['USER_ID', 'ITEM_ID'], 
                            how='outer', 
                            indicator=True)
df_train = df_train[(df_train['_merge'] == 'left_only')]

df_train


,USER_ID,ITEM_ID,RATING,TIMESTAMP,_USER_IDX,_ITEM_IDX,_merge
0,196,242,3.0,881250949,646,287,left_only
1,305,242,5.0,886307828,121,287,left_only
2,6,242,4.0,883268170,134,287,left_only
3,234,242,4.0,891033261,8,287,left_only
4,63,242,3.0,875747190,378,287,left_only
...,...,...,...,...,...,...,...
99251,436,1468,5.0,887770668,243,1269,left_only
99254,405,1474,1.0,885547645,0,1270,left_only
99260,405,1522,1.0,885548670,0,1257,left_only
99268,409,1558,5.0,881107281,145,1244,left_only


In [10]:
helper.put_csv(df_train, dataset_name, "interactions_train.csv.gz")

